In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('backorder').getOrCreate()

In [32]:
from pyspark.ml.classification import LogisticRegression

In [4]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType, DoubleType
from pyspark.ml.feature import StringIndexer

In [5]:
train = spark.read.csv("Kaggle_Test_Dataset_v2.csv", inferSchema = True, header = True)

In [6]:
train.printSchema()

root
 |-- sku: string (nullable = true)
 |-- national_inv: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- in_transit_qty: integer (nullable = true)
 |-- forecast_3_month: integer (nullable = true)
 |-- forecast_6_month: integer (nullable = true)
 |-- forecast_9_month: integer (nullable = true)
 |-- sales_1_month: integer (nullable = true)
 |-- sales_3_month: integer (nullable = true)
 |-- sales_6_month: integer (nullable = true)
 |-- sales_9_month: integer (nullable = true)
 |-- min_bank: integer (nullable = true)
 |-- potential_issue: string (nullable = true)
 |-- pieces_past_due: integer (nullable = true)
 |-- perf_6_month_avg: double (nullable = true)
 |-- perf_12_month_avg: double (nullable = true)
 |-- local_bo_qty: integer (nullable = true)
 |-- deck_risk: string (nullable = true)
 |-- oe_constraint: string (nullable = true)
 |-- ppap_risk: string (nullable = true)
 |-- stop_auto_buy: string (nullable = true)
 |-- rev_stop: string (nullable = true)
 |-- 

In [7]:
train.groupBy('went_on_backorder').count().show()


+-----------------+------+
|went_on_backorder| count|
+-----------------+------+
|             null|     1|
|               No|239387|
|              Yes|  2688|
+-----------------+------+



In [8]:
train = train.dropna(subset = 'went_on_backorder')
train.groupBy('went_on_backorder').count().show()


+-----------------+------+
|went_on_backorder| count|
+-----------------+------+
|               No|239387|
|              Yes|  2688|
+-----------------+------+



In [9]:
indexer = StringIndexer(inputCol="went_on_backorder", outputCol="backorder")

In [10]:
#df4.na.replace(['Alice', 'Bob'], ['A', 'B'], 'name').show()

#train = train.na.replace(['Yes', 'No'], ['1', '0'], 'went_on_backorder')
#train.printSchema()

#train = train.withColumn('backorder', train['went_on_backorder']==1)
#train = train.withColumn("backorder", train["backorder"].cast(DoubleType()))
#train.printSchema()

root
 |-- sku: string (nullable = true)
 |-- national_inv: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- in_transit_qty: integer (nullable = true)
 |-- forecast_3_month: integer (nullable = true)
 |-- forecast_6_month: integer (nullable = true)
 |-- forecast_9_month: integer (nullable = true)
 |-- sales_1_month: integer (nullable = true)
 |-- sales_3_month: integer (nullable = true)
 |-- sales_6_month: integer (nullable = true)
 |-- sales_9_month: integer (nullable = true)
 |-- min_bank: integer (nullable = true)
 |-- potential_issue: string (nullable = true)
 |-- pieces_past_due: integer (nullable = true)
 |-- perf_6_month_avg: double (nullable = true)
 |-- perf_12_month_avg: double (nullable = true)
 |-- local_bo_qty: integer (nullable = true)
 |-- deck_risk: string (nullable = true)
 |-- oe_constraint: string (nullable = true)
 |-- ppap_risk: string (nullable = true)
 |-- stop_auto_buy: string (nullable = true)
 |-- rev_stop: string (nullable = true)
 |-- 

In [10]:
train = train.withColumn("national_inv", train["national_inv"].cast(DoubleType()))
train = train.withColumn("lead_time", train["lead_time"].cast(DoubleType()))
train = train.withColumn("in_transit_qty", train["in_transit_qty"].cast(DoubleType()))
train = train.withColumn("min_bank", train["min_bank"].cast(DoubleType()))
train = train.withColumn("local_bo_qty", train["local_bo_qty"].cast(DoubleType()))


In [11]:
train.printSchema()

root
 |-- sku: string (nullable = true)
 |-- national_inv: double (nullable = true)
 |-- lead_time: double (nullable = true)
 |-- in_transit_qty: double (nullable = true)
 |-- forecast_3_month: integer (nullable = true)
 |-- forecast_6_month: integer (nullable = true)
 |-- forecast_9_month: integer (nullable = true)
 |-- sales_1_month: integer (nullable = true)
 |-- sales_3_month: integer (nullable = true)
 |-- sales_6_month: integer (nullable = true)
 |-- sales_9_month: integer (nullable = true)
 |-- min_bank: double (nullable = true)
 |-- potential_issue: string (nullable = true)
 |-- pieces_past_due: integer (nullable = true)
 |-- perf_6_month_avg: double (nullable = true)
 |-- perf_12_month_avg: double (nullable = true)
 |-- local_bo_qty: double (nullable = true)
 |-- deck_risk: string (nullable = true)
 |-- oe_constraint: string (nullable = true)
 |-- ppap_risk: string (nullable = true)
 |-- stop_auto_buy: string (nullable = true)
 |-- rev_stop: string (nullable = true)
 |-- went_

In [12]:
train = train.fillna(0, subset = 'lead_time')
train.groupBy('lead_time').count().show()

+---------+-----+
|lead_time|count|
+---------+-----+
|      8.0|98533|
|      0.0|16239|
|      7.0|   30|
|     35.0|    5|
|     18.0|   32|
|      1.0|    3|
|     25.0|    1|
|      4.0|18508|
|     23.0|    2|
|     11.0|  152|
|     21.0|    7|
|     14.0| 1506|
|     22.0|   19|
|      3.0| 2349|
|     19.0|    4|
|     28.0|   12|
|      2.0|46917|
|     17.0|  537|
|     10.0| 2056|
|     40.0|    8|
+---------+-----+
only showing top 20 rows



In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(inputCols = ["national_inv", "lead_time", "in_transit_qty", "min_bank", "local_bo_qty",
                                        "perf_6_month_avg", "perf_12_month_avg"],
                           outputCol = "features")

In [15]:
output = assembler.transform(train)

In [16]:
outputIndexed = indexer.fit(output).transform(output)

In [18]:
outputIndexed.select("features", "backorder").show(truncate = False)

+----------------------------------+---------+
|features                          |backorder|
+----------------------------------+---------+
|[62.0,0.0,0.0,1.0,0.0,-99.0,-99.0]|0.0      |
|[9.0,0.0,0.0,1.0,0.0,-99.0,-99.0] |0.0      |
|[17.0,8.0,0.0,0.0,0.0,0.92,0.95]  |0.0      |
|[9.0,2.0,0.0,0.0,0.0,0.78,0.75]   |0.0      |
|[2.0,8.0,0.0,0.0,0.0,0.54,0.71]   |0.0      |
|[15.0,2.0,0.0,0.0,0.0,0.37,0.68]  |0.0      |
|(7,[5,6],[-99.0,-99.0])           |0.0      |
|(7,[0,5,6],[28.0,-99.0,-99.0])    |0.0      |
|(7,[0,5,6],[2.0,-99.0,-99.0])     |0.0      |
|(7,[0,5,6],[2.0,-99.0,-99.0])     |0.0      |
|[20.0,0.0,0.0,1.0,0.0,-99.0,-99.0]|0.0      |
|(7,[5,6],[-99.0,-99.0])           |0.0      |
|(7,[0,5,6],[13.0,-99.0,-99.0])    |0.0      |
|[208.0,16.0,0.0,0.0,0.0,0.66,0.64]|0.0      |
|(7,[5,6],[-99.0,-99.0])           |0.0      |
|(7,[1,5,6],[2.0,0.37,0.34])       |0.0      |
|(7,[5,6],[-99.0,-99.0])           |0.0      |
|(7,[0,5,6],[26.0,-99.0,-99.0])    |0.0      |
|(7,[0,5,6],[

In [19]:
data = outputIndexed.select("features", "backorder")

In [20]:
train_data, test_data = data.randomSplit([0.9, 0.1])

In [33]:
lr = LogisticRegression(labelCol = "backorder")

In [34]:
lrModel = lr.fit(train_data)

In [35]:
trainingModel = lrModel.summary

In [36]:
trainingModel.predictions.show()

+---------+---------+--------------------+--------------------+----------+
| features|backorder|       rawPrediction|         probability|prediction|
+---------+---------+--------------------+--------------------+----------+
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|(7,[],[])|      0.0|[3.9

In [37]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [38]:
predictionAndLables = lrModel.evaluate(test_data)

In [39]:
predictionAndLables.predictions.show()


+-------------------+---------+--------------------+--------------------+----------+
|           features|backorder|       rawPrediction|         probability|prediction|
+-------------------+---------+--------------------+--------------------+----------+
|          (7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|          (7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|          (7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|          (7,[],[])|      0.0|[3.93866514404500...|[0.98089780713855...|       0.0|
|      (7,[0],[2.0])|      0.0|[3.94417645822433...|[0.98100080104712...|       0.0|
|      (7,[0],[5.0])|      0.0|[3.95244342949331...|[0.98115427161655...|       0.0|
|      (7,[0],[6.0])|      0.0|[3.95519908658298...|[0.98120515777637...|       0.0|
|     (7,[0],[19.0])|      0.0|[3.99102262874859...|[0.98185453731543...|       0.0|
|     (7,[0],[25.0])|      0.0|[4.00755657128656...|[0.9821467745

In [40]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='backorder')

In [41]:
auc = churn_eval.evaluate(predictionAndLables.predictions)
auc

0.5

In [44]:
from pyspark.sql.functions import corr

In [45]:
train.select(corr('went_on_backorder','national_inv')).show()

+-------------------------------------+
|corr(went_on_backorder, national_inv)|
+-------------------------------------+
|                 -0.00173416496506...|
+-------------------------------------+



In [46]:
train.select(corr('went_on_backorder','lead_time')).show()

+----------------------------------+
|corr(went_on_backorder, lead_time)|
+----------------------------------+
|              -0.01729943162873909|
+----------------------------------+



In [47]:
train.select(corr('went_on_backorder','in_transit_qty')).show()

+---------------------------------------+
|corr(went_on_backorder, in_transit_qty)|
+---------------------------------------+
|                   -0.00351683289259...|
+---------------------------------------+



In [48]:
train.select(corr('went_on_backorder','min_bank')).show()

+---------------------------------+
|corr(went_on_backorder, min_bank)|
+---------------------------------+
|             -0.00102680463935...|
+---------------------------------+



In [49]:

train.select(corr('went_on_backorder','local_bo_qty')).show()

+-------------------------------------+
|corr(went_on_backorder, local_bo_qty)|
+-------------------------------------+
|                 0.006355025990898093|
+-------------------------------------+



In [50]:
train.select(corr('went_on_backorder','perf_6_month_avg')).show()

+-----------------------------------------+
|corr(went_on_backorder, perf_6_month_avg)|
+-----------------------------------------+
|                      0.01266287406108172|
+-----------------------------------------+



In [51]:
train.select(corr('went_on_backorder','perf_12_month_avg')).show()

+------------------------------------------+
|corr(went_on_backorder, perf_12_month_avg)|
+------------------------------------------+
|                      0.012802138240801638|
+------------------------------------------+



In [52]:
train.select(corr('went_on_backorder','sales_9_month')).show()

+--------------------------------------+
|corr(went_on_backorder, sales_9_month)|
+--------------------------------------+
|                  -0.00188847244692...|
+--------------------------------------+



In [55]:

train.select(corr('sales_9_month','forecast_9_month')).show()

+-------------------------------------+
|corr(sales_9_month, forecast_9_month)|
+-------------------------------------+
|                   0.8606098866748187|
+-------------------------------------+



In [56]:
train.select(corr('sales_6_month','forecast_6_month')).show()

+-------------------------------------+
|corr(sales_6_month, forecast_6_month)|
+-------------------------------------+
|                   0.7955021547041715|
+-------------------------------------+



In [58]:
train.select(corr('went_on_backorder','min_bank')).show()

+---------------------------------+
|corr(went_on_backorder, min_bank)|
+---------------------------------+
|             -0.00102680463935...|
+---------------------------------+



In [59]:
train.select(corr('went_on_backorder','pieces_past_due')).show()

+----------------------------------------+
|corr(went_on_backorder, pieces_past_due)|
+----------------------------------------+
|                    0.001513612185315...|
+----------------------------------------+



In [60]:

train.select(corr('went_on_backorder','local_bo_qty')).show()

+-------------------------------------+
|corr(went_on_backorder, local_bo_qty)|
+-------------------------------------+
|                 0.006355025990898093|
+-------------------------------------+



In [61]:
train.select(corr('went_on_backorder','went_on_backorder')).show()

+------------------------------------------+
|corr(went_on_backorder, went_on_backorder)|
+------------------------------------------+
|                                       1.0|
+------------------------------------------+



In [ ]:
### Use Gradient Boosted Trees

In [21]:
from pyspark.ml.classification import GBTClassifier

In [22]:
gbt = GBTClassifier(labelCol="backorder", featuresCol="features", maxIter=10)

In [23]:
model = gbt.fit(train_data)

In [24]:
predictions = model.transform(test_data)

In [30]:
predictions.select("prediction", "backorder", "features").show(100)

+----------+---------+--------------------+
|prediction|backorder|            features|
+----------+---------+--------------------+
|       0.0|      0.0|           (7,[],[])|
|       0.0|      0.0|           (7,[],[])|
|       0.0|      0.0|           (7,[],[])|
|       0.0|      0.0|           (7,[],[])|
|       0.0|      0.0|       (7,[0],[2.0])|
|       0.0|      0.0|       (7,[0],[5.0])|
|       0.0|      0.0|       (7,[0],[6.0])|
|       0.0|      0.0|      (7,[0],[19.0])|
|       0.0|      0.0|      (7,[0],[25.0])|
|       0.0|      0.0|      (7,[0],[57.0])|
|       0.0|      0.0|     (7,[0],[151.0])|
|       0.0|      0.0|     (7,[0],[213.0])|
|       0.0|      0.0|     (7,[0],[236.0])|
|       0.0|      0.0|     (7,[0],[307.0])|
|       0.0|      0.0|     (7,[0],[383.0])|
|       0.0|      0.0| (7,[0,1],[1.0,8.0])|
|       0.0|      0.0| (7,[0,1],[1.0,8.0])|
|       0.0|      0.0| (7,[0,1],[1.0,8.0])|
|       0.0|      0.0| (7,[0,1],[1.0,8.0])|
|       0.0|      0.0| (7,[0,1],

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="backorder", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0116519


In [29]:
accuracy

0.9883480704074044